# Class and Object

## 改变对象的字符串显示

你想改变对象实例的打印或者显示输出，要改变一个实例的字符串表示，可重新定义它的str repr方法

In [11]:
class Pair:
    def __init__(self, x, y):
        self.x = x 
        self.y = y
        
    def __repr__(self): # 返回一个实例的代码表示形式，通常用来重新构造这个实例
        return 'Pair ({0.x!r}, {0.y!r})'.format(self) # 0.x 0.y self.x self.y
    
    def __str__(self): # 将实例转换为一个字符串
        return '({0.x!s}, {0.y!s})'.format(self)
    
    '''
    def __repr__(self):
        return 'Pair(%s, %s') % (self.x, self.y) # instead
    '''
    
p = Pair(3, 4)
p

Pair (3, 4)

In [6]:
print(p)

(3, 4)


In [8]:
print('p is {!r}'.format(p))

p is Pair (3, 4)


In [9]:
print('p is {}'.format(p))

p is (3, 4)


自定义 __repr__() 和 __str__() 通常是很好的习惯，因为它能简化调试和实例输出。 例如，如果仅仅只是打印输出或日志输出某个实例，那么程序员会看到实例更加详细与有用的信息

In [10]:
print('debug : p is [{0!r}]')

debug : p is [{0!r}]


## 自定义字符串的格式化

In [29]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

_formats_1 = {
    'ymd' : '{0.year}-{0.month}-{0.day}',
    'mdy' : '{0.month}/{0.day}/{0.year}',
    'dmy' : '{0.day}/{0.month}/{0.year}'
    }

class Date:
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)
    
d = Date(2012, 12, 21)

In [19]:
format(d) # format Method from ?

'2012-12-21'

In [22]:
format(d, 'mdy'),

('12/21/2012',)

In [23]:
'the date is {:ymd}'.format(d), 'the date is {:mdy}'.format(d)

('the date is 2012-12-21', 'the date is 12/21/2012')

In [26]:
#self-1
_formats_1 = {
    'ymd' : '{0.year}-{0.month}-{0.day}',
    'mdy' : '{0.month}/{0.day}/{0.year}',
    'dmy' : '{0.day}/{0.month}/{0.year}'
    }

class Date:
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats_1[code]
        return fmt.format(self)
    
d = Date(2012, 12, 21)

In [28]:
format(d), format(d, 'mdy'), 'the date is {:ymd}'.format(d), 'the date is {:mdy}'.format(d)

('2012-12-21',
 '12/21/2012',
 'the date is 2012-12-21',
 'the date is 12/21/2012')

__format__() 方法给Python的字符串格式化功能提供了一个钩子。 这里需要着重强调的是格式化代码的解析工作完全由类自己决定。因此，格式化代码可以是任何值。 例如，参考下面来自 datetime 模块中的代码

In [30]:
from datetime import date

d = date(2012, 12, 21)
format(d)

'2012-12-21'

In [31]:
format(d, '%A %B %d %Y')

'Friday December 21 2012'

In [32]:
'the end is {:%d %b %Y}. Goodbye'.format(d)

'the end is 21 Dec 2012. Goodbye'

## 让对象支持上下文管理协议

你想让你的对象支持上下文管理协议(with语句)

为了让一个对象兼容 with 语句，你需要实现 __enter__() 和 __exit__() 方法。 例如，考虑如下的一个类，它能为我们创建一个网络连接

In [34]:
# 这个类的关键特点在于它表示了一个网络连接，
# 但是初始化的时候并不会做任何事情(比如它并没有建立一个连接)。 连接的建立和关闭是使用with语句自动完成的
from socket import socket, AF_INET, SOCK_STREAM

class Lazyconnection:
    
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None
        
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [35]:
from functools import partial

conn = Lazyconnection(('www.python.org', 80)) # (())? address = ('www.python.org', 80) 
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes:connection closed

In [36]:
resp

b'HTTP/1.1 301 Moved Permanently\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nContent-Length: 0\r\nAccept-Ranges: bytes\r\nDate: Sat, 03 Aug 2019 08:19:22 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\nX-Served-By: cache-hnd18746-HND\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1564820362.247628,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains\r\n\r\n'

编写上下文管理器的主要原理是你的代码会放到 with 语句块中执行。 当出现 with 语句的时候，对象的 __enter__() 方法被触发， 它返回的值(如果有的话)会被赋值给 as 声明的变量。然后，with 语句块里面的代码开始执行。 最后，__exit__() 方法被触发进行清理工作

不管 with 代码块中发生什么，上面的控制流都会执行完，就算代码块中发生了异常也是一样的。 事实上，__exit__() 方法的第三个参数包含了异常类型、异常值和追溯信息(如果有的话)。 __exit__() 方法能自己决定怎样利用这个异常信息，或者忽略它并返回一个None值。 如果 __exit__() 返回 True ，那么异常会被清空，就好像什么都没发生一样， with 语句后面的程序继续在正常执行

In [38]:
from socket import socket, AF_INET, SOCK_STREAM

class Lazyconnection:
    
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        #self.sock = None
        self.connections = []
        
    def __enter__(self):
        #if self.sock is not None:
        #    raise RuntimeError('Already connected')
        sock = socket(self.family, self.type)
        #self.sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connection.append(sock)
        return sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        #self.sock.close()
        #self.sock = None
        self.connections.pop().close()

In [39]:
# use case

from functools import partial

conn = Lazyconnection(('www.python.org', 8))
with conn as s1:
    pass
    with conn as s2:
        pass

TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

在第二个版本中，LazyConnection 类可以被看做是某个连接工厂。在内部，一个列表被用来构造一个栈。 每次 __enter__() 方法执行的时候，它复制创建一个新的连接并将其加入到栈里面。 __exit__() 方法简单的从栈中弹出最后一个连接并关闭它。 这里稍微有点难理解，不过它能允许嵌套使用 with 语句创建多个连接，就如上面演示的那样。

在需要管理一些资源比如文件、网络连接和锁的编程环境中，使用上下文管理器是很普遍的。 这些资源的一个主要特征是它们必须被手动的关闭或释放来确保程序的正确运行。 例如，如果你请求了一个锁，那么你必须确保之后释放了它，否则就可能产生死锁。 通过实现 __enter__() 和 __exit__() 方法并使用 with 语句可以很容易的避免这些问题， 因为 __exit__() 方法可以让你无需担心这些了。

在 contextmanager 模块中有一个标准的上下文管理方案模板，可参考9.22小节。 同时在12.6小节中还有一个对本节示例程序的线程安全的修改版

## 创建大量对象时节省内存方法

你的程序要创建大量(可能上百万)的对象，导致占用很大的内存

对于主要是用来当成简单的数据结构的类而言，你可以通过给类添加 __slots__ 属性来极大的减少实例所占的内存

In [41]:
class Date:
    __slots__ = ['year', 'month', 'day'] # ? 
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

当你定义 __slots__ 后，Python就会为实例使用一种更加紧凑的内部表示。 实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典，这跟元组或列表很类似。 在 __slots__ 中列出的属性名在内部被映射到这个数组的指定小标上。 使用slots一个不好的地方就是我们不能再给实例添加新的属性了，只能使用在 __slots__ 中定义的那些属性名

使用slots后节省的内存会跟存储属性的数量和类型有关。 不过，一般来讲，使用到的内存总量和将数据存储在一个元组中差不多。 为了给你一个直观认识，假设你不使用slots直接存储一个Date实例， 在64位的Python上面要占用428字节，而如果使用了slots，内存占用下降到156字节。 如果程序中需要同时创建大量的日期实例，那么这个就能极大的减小内存使用量了。

尽管slots看上去是一个很有用的特性，很多时候你还是得减少对它的使用冲动。 Python的很多特性都依赖于普通的基于字典的实现。 另外，定义了slots后的类不再支持一些普通类特性了，比如多继承。 大多数情况下，你应该只在那些经常被使用到的用作数据结构的类上定义slots (比如在程序中需要创建某个类的几百万个实例对象)。

关于 __slots__ 的一个常见误区是它可以作为一个封装工具来防止用户给实例增加新的属性。 尽管使用slots可以达到这样的目的，但是这个并不是它的初衷。 __slots__ 更多的是用来作为一个内存优化工具

## 在类中封装属性名

你想封装类的实例上面的“私有”数据，但是Python语言并没有访问控制

Python程序员不去依赖语言特性去封装数据，而是通过遵循一定的属性和方法命名规约来达到这个效果。 第一个约定是任何以单下划线_开头的名字都应该是内部实现

In [42]:
class A:
    
    def __init__(self):
        self._internal = 0 # an internal attribute
        self.public = 1 # a public attribute
        
    def pubilic_method(self):
        pass
    
    def _internal_method(self):
        pass

Python并不会真的阻止别人访问内部名称。但是如果你这么做肯定是不好的，可能会导致脆弱的代码。 同时还要注意到，使用下划线开头的约定同样适用于模块名和模块级别函数。 例如，如果你看到某个模块名以单下划线开头(比如_socket)，那它就是内部实现。 类似的，模块级别函数比如 sys._getframe() 在使用的时候就得加倍小心了。

你还可能会遇到在类定义中使用两个下划线(__)开头的命名

In [44]:
class B:
    
    def __init__(self):
        self.__private = 0
        
    def __private_method(self):
        pass
    
    def public_method(self):
        pass
        self.__private_method()

In [ ]:
#使用双下划线开始会导致访问名称变成其他形式。 
#比如，在前面的类B中，私有属性会被分别重命名为 _B__private 和 #_B__private_method
#这时候你可能会问这样重命名的目的是什么，答案就是继承——这种属性通过继承是无法被覆盖

class C(B):
    def __init__(self):
        super().__init__() # ?
        self.__private = 1 # Does not override B.__private
        
    # Dose  not override B.__private_method()
    def __private_method(self):
        pass
    
# 私有名称 __private 和 __private_method 被重命名为 _C__private 和 _C__private_method
#这个跟父类B中的名称是完全不同的

你应该让你的非公共名称以单下划线开头。但是，如果你清楚你的代码会涉及到子类， 并且有些内部属性应该在子类中隐藏起来，那么才考虑使用双下划线方案

## 创建可管理的属性

你想给某个实例attribute增加除访问与修改之外的其他处理逻辑，比如类型检查或合法性验证

自定义某个属性的一种简单方法是将它定义为一个property。 例如，下面的代码定义了一个property，增加对一个属性简单的类型检查

In [48]:
class Person:
    
    def __init__(self,first_name):
        self.first_name = first_name
        
    # Getter function
    @property
    def first_name(self):
        return self._first_name # self._first_name?
    
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError('Can"t delete attribute')
        
# 上述代码中有三个相关联的方法，这三个方法的名字都必须一样。 
# 第一个方法是一个 getter 函数，它使得 first_name 成为一个属性。 
# 其他两个方法给 first_name 属性添加了 setter 和 deleter 函数。 
# 需要强调的是只有在 first_name 属性被创建后， 后面的两个装饰器 @first_name.setter 和 @first_name.deleter 才能被定义

In [49]:
a = Person('Guido')
a.first_name

'Guido'

In [50]:
a.first_name = 42

TypeError: Expected a string

In [51]:
del a.first_name

AttributeError: Can"t delete attribute

In [54]:
#在实现一个property的时候，底层数据(如果有的话)仍然需要存储在某个地方。 
#因此，在get和set方法中，你会看到对 _first_name 属性的操作，这也是实际数据保存的地方。 
#另外，你可能还会问为什么 __init__() 方法中设置了 (self.first_name) 而不是 (self._first_name) 。 
#在这个例子中，我们创建一个property的目的就是在设置attribute的时候进行检查。 
#因此，你可能想在初始化的时候也进行这种类型检查。
#通过设置 self.first_name ，自动调用 setter 方法， 这个方法里面会进行参数的检查，否则就是直接访问 self._first_name了

# 还能在已存在的get和set方法基础上定义property